## Example for SDK

This example shows how to use SDK to deploy a task (V2)

### Initialization

For test version, get a user `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io/provider-status

In [1]:
import os
import time
import dotenv
dotenv.load_dotenv("../.env")
from swan import SwanAPI , MCSAPI

# Initialize the Swan Service
swan_api = SwanAPI(api_key=os.getenv("API_KEY"), environment="https://swanhub-cali.swanchain.io")

api_key = os.getenv("MCS_API_KEY")
mcs_api = MCSAPI(api_key)

### Available hardware information

In [2]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
# hardwares_info

choose hardware config

In [3]:
device = 'C1ae.medium' #"G1ae.medium"
obj = [hardware for hardware in hardwares if hardware.name == device][0]
print(obj.id)
print([(hardware.name, hardware.id, hardware.region) for hardware in hardwares if hardware.name == device][0])

1
('C1ae.medium', 1, ['North Carolina-US', 'Quebec-CA'])


to simplify the process, here we use a existing `job_source_uri` which is a hello world application, used to create task.

In [4]:
job_source_uri = 'https://test-api.lagrangedao.org/spaces/5117e998-c623-4837-8af9-2b7b0ce2de7f'

### Define task deploy v2

In [5]:
# Deploy task

# before v2 integrated into SDK, use customed function instead
import logging
import traceback
import json
from swan.common.constant import *
from swan.common.exception import SwanAPIException

def deploy_task_v2(
        cfg_name: str, 
        region: str, 
        start_in: int, 
        duration: int, 
        job_source_uri: str, 
        wallet_address: str, 
        paid: float = 0.0
    ):
    """Sent deploy task request via orchestrator.

    Args:
        cfg_name: name of cp/hardware configuration set.
        region: region of hardware.
        start_in: unix timestamp of starting time.
        duration: duration of service runtime in unix time.
        job_source_uri: source uri for space.
        wallet_address: user wallet address.
        paid: paid amount in Eth.

    Returns:
        JSON response from backend server including 'task_uuid'.
    """
    try:
        if swan_api._verify_hardware_region(cfg_name, region):
            params = {
                "paid": paid,
                "duration": duration,
                "cfg_name": cfg_name,
                "region": region,
                "start_in": start_in,
                "wallet": wallet_address,
                "job_source_uri": job_source_uri
            }
            result = swan_api._request_with_params(
                POST, 
                '/v2/task_deployment', 
                swan_api.swan_url, 
                params, 
                swan_api.token, 
                None
            )
            return result
        else:
            raise SwanAPIException(f"No {cfg_name} machine in {region}.")
    except Exception as e:
        logging.error(str(e) + traceback.format_exc())
        return None


### Define contract util class v2

In [6]:
# ./swan/contract/swan_contract_ex.py


from swan.common.constant import *
from swan.common.utils import get_contract_abi

from swan.contract.swan_contract import SwanContract


CLIENT_CONTRACT_ADDRESS="0xe356a758fA1748dfBE71E989c876959665a66ddA"
_CLIENT_CONTRACT_ABI = "../swan/contract/abi/ClientPayment.json"

class SwanContractEx(SwanContract):

    def __init__(self, private_key: str, rpc_url: str):
        """ Initialize swan contract API connection.

        Args:
            private_key: private key for wallet.
            rpc_url: rpc url of swan chain for connection.
        """

        # with open(_CLIENT_CONTRACT_ABI, 'r') as abi_file:
        #     abi_data = json.load(abi_file)
        #     client_abi = json.dumps(abi_data)
        
        
        super().__init__(private_key=private_key, rpc_url=rpc_url)
        self.client_contract = self.w3.eth.contract(
            address=CLIENT_CONTRACT_ADDRESS, 
            abi=json.load(open(_CLIENT_CONTRACT_ABI))
        )

    def submit_payment(self, task_id: str, hardware_id: int, duration: int):
        nonce = self.w3.eth.get_transaction_count(self.account.address)
        base_fee = self.w3.eth.get_block('latest')['baseFeePerGas']
        max_priority_fee_per_gas = self.w3.to_wei(2, 'gwei')
        max_fee_per_gas = base_fee + max_priority_fee_per_gas
        if max_fee_per_gas < max_priority_fee_per_gas:
            max_fee_per_gas = max_priority_fee_per_gas + base_fee
        tx = self.client_contract.functions.submitPayment(task_id, hardware_id, duration).build_transaction({
            'from': self.account.address,
            'nonce': nonce,
            "maxFeePerGas": max_fee_per_gas,
            "maxPriorityFeePerGas": max_priority_fee_per_gas,
        })
        signed_tx = self.w3.eth.account.sign_transaction(tx, self.account._private_key)
        tx_hash = self.w3.eth.send_raw_transaction(signed_tx.rawTransaction)
        self.w3.eth.wait_for_transaction_receipt(tx_hash, timeout=CONTRACT_TIMEOUT)
        return self.w3.to_hex(tx_hash)
    
    def _approve_swan_token(self, amount):
        nonce = self.w3.eth.get_transaction_count(self.account.address)
        base_fee = self.w3.eth.get_block('latest')['baseFeePerGas']
        max_priority_fee_per_gas = self.w3.to_wei(2, 'gwei')
        max_fee_per_gas = base_fee + max_priority_fee_per_gas
        if max_fee_per_gas < max_priority_fee_per_gas:
            max_fee_per_gas = max_priority_fee_per_gas + base_fee
        tx = self.token_contract.functions.approve(self.client_contract.address, amount).build_transaction({
            'from': self.account.address,
            'nonce': nonce,
            "maxFeePerGas": max_fee_per_gas,
            "maxPriorityFeePerGas": max_priority_fee_per_gas,
        })
        signed_tx = self.w3.eth.account.sign_transaction(tx, self.account._private_key)
        tx_hash = self.w3.eth.send_raw_transaction(signed_tx.rawTransaction)
        self.w3.eth.wait_for_transaction_receipt(tx_hash, timeout=CONTRACT_TIMEOUT)
        return self.w3.to_hex(tx_hash)
    

### Contract and Payment Estimation

- Firstly, use contract function to estimate the amount to pay. 
- Secondly, by `task_uuid` gotten from v2 API `/v2/task_deployment`, **pay** with `task_uuid` and `hardware_id`
- Thirdly, do payment validation via v2 API `/v2/task_payment_validate`, which will enable task eligible for assigning

In [7]:
pk = os.getenv('PK')
rpc = os.getenv('RPC')

c2 = SwanContractEx(pk, rpc)
duration_hour = 1 # hour
amount = c2.estimate_payment(obj.id, duration_hour)
print(amount)

1000000000000000000


### Deploy task

This step shows how to use SDK's interface for deploying task, which calls Orchestrator's task deployment API (V2), to get `task_uuid`, which will be used in payment.

In [8]:
duration=3600*duration_hour

result = deploy_task_v2(
            cfg_name=device, 
            region='Quebec-CA', 
            start_in=5, 
            duration=duration, 
            job_source_uri=job_source_uri, 
            paid=c2._wei_to_swan(amount),
            wallet_address=os.getenv('WALLET'),
          )
print(result)
task_uuid = result['data']['task']['uuid']
print("Task UUID:", task_uuid)

{'data': {'task': {'created_at': '1713377715', 'end_at': '1713381312', 'leading_job_id': None, 'refund_amount': None, 'status': 'initialized', 'task_detail_cid': 'https://plutotest.acl.swanipfs.com/ipfs/QmUvHQNGETErt6MP7PMx6525ZgKKoe19zyt6oBkVR9kvzU', 'tx_hash': None, 'updated_at': '1713377715', 'uuid': '0242bdf6-45bb-4743-b081-f86d4f702bc0'}}, 'message': 'Task_uuid initialized.', 'status': 'success'}
Task UUID: 0242bdf6-45bb-4743-b081-f86d4f702bc0


### Submit Payment

This step is using `task_uuid`, `hardware_id` and `duration` to submit payment via **ClientPayment** contract.

In [9]:
r = c2._approve_swan_token(amount)
print(r)
 
tx_hash = c2.submit_payment(task_uuid, obj.id, duration)
print(tx_hash)

0x30b0bce0f724fc7fc56e89c2e79c520cc68b5376246212d64b8427b25de6138e
0x4fcabb68f20381cc8f10fe5873e7c544ec0500fb4b84542230e21db803fd1336


### Validate Payment via API

This step will validate the payment and then make task eligible for assigning if validation successful

In [31]:

def validate_payment(
        tx_hash,
        task_uuid
    ):
        
    print(tx_hash)
    print(task_uuid)

    try:
        if tx_hash and task_uuid:
            params = {
                "tx_hash": tx_hash,
                "task_uuid": task_uuid
            }
            print(params)
            result = swan_api._request_with_params(
                POST, 
                '/v2/task_payment_validate', 
                swan_api.swan_url, 
                params, 
                os.getenv("API_KEY"), #swan_api.token, 
                None
            )
            return result
        else:
            raise SwanAPIException(f"{tx_hash=} or {task_uuid=} invalid")
    except Exception as e:
        logging.error(str(e) + traceback.format_exc())
        return None

result_validation = validate_payment(
    tx_hash=tx_hash,
    task_uuid=task_uuid
)
print(result_validation)

0x70d958a1bb0d43b88215674789dbdf142b9d3bb512c1a5345dd8e09dcb6c09ce
b35d5ce8-2dcb-4e67-88de-f48d89487418
{'tx_hash': '0x70d958a1bb0d43b88215674789dbdf142b9d3bb512c1a5345dd8e09dcb6c09ce', 'task_uuid': 'b35d5ce8-2dcb-4e67-88de-f48d89487418'}
{'data': {'error_code': 1131}, 'message': 'payment validation failed: payment receipt contract address is not correct', 'status': 'failed'}


The following step is optional, shows information when waiting for task being deployed.

In [9]:
# Check task info
while True:
    info = swan_api.get_deployment_info(task_uuid=task_uuid)
    if len(info['data']['jobs']) > 0:
        
        status = info['data']['jobs'][0]['status']
        print(status)
        
        job_res_uri = info['data']['jobs'][0]['job_result_uri']
        job_real_uri = info['data']['jobs'][0]['job_real_uri']
        print("Job Result URL: ", job_res_uri)
        print("Job Real URL: ", job_real_uri)
        
        # break
        if status == 'deployToK8s' or status == "Cancelled" or status == "Failed":
            break
        
    time.sleep(30)

buildImage
Job Result URL:  https://42f6d9f62851.acl.swanipfs.com/ipfs/QmQEW9Dyi9aEQYtVdMxA7qkr7gdiiNE9MJxNtjt9uSyUKc
Job Real URL:  https://cf24tc2oot.dev2.crosschain.computer
deployToK8s
Job Result URL:  https://42f6d9f62851.acl.swanipfs.com/ipfs/QmQEW9Dyi9aEQYtVdMxA7qkr7gdiiNE9MJxNtjt9uSyUKc
Job Real URL:  https://cf24tc2oot.dev2.crosschain.computer


### Show result

`job_real_uri` is for show the result of application you deployed.

In [10]:
r = swan_api.get_real_url(task_uuid)
print(r)

ERROR:root:'NoneType' object is not subscriptableTraceback (most recent call last):
  File "/Users/aaronli/miniconda3/envs/test-sdk2/lib/python3.10/site-packages/swan/api/swan_api.py", line 184, in get_real_url
    jobs = deployment_info['data']['jobs']
TypeError: 'NoneType' object is not subscriptable



None


In [12]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


{
    "Hello": "World! Today - 06.21"
}
